In [ ]:
# 1) Imports
import joblib
import pandas as pd
import matplotlib.pyplot as plt

# 2) Load artifacts (sube un nivel porque estamos en notebooks/)
tfidf = joblib.load('../models/tfidf_vectorizer.joblib')
model = joblib.load('../models/final_fake_news_model.joblib')

# 3) Function to classify new texts
def classify_texts(texts, threshold=0.7):
    """
    texts: list of strings
    threshold: probability cutoff for labeling 'real' vs 'fake'
    """
    X     = tfidf.transform(texts)
    probs = model.predict_proba(X)[:, 1]
    preds = (probs >= threshold).astype(int)
    for txt, p, pr in zip(texts, probs, preds):
        label = 'real' if pr == 1 else 'fake'
        print(f"📰 {txt}\n    → Predicted: {label} (p_real={p:.2f}, p_fake={(1-p):.2f})\n")

# 4) Example headlines to test
to_check = [
    "Breaking news: Scientists discover a new species in the Amazon rainforest.",
    "Local health officials report that vaccination rates have reached a record high this quarter.",
    "In an exclusive interview, the celebrity reveals her secret to eternal youth."
]

# 5) Classify the example headlines with threshold = 0.7
print("---- Test provided examples ----")
classify_texts(to_check, threshold=0.7)

# 6) Verify on a true‐news sample from the dataset
real_df      = pd.read_csv('../data/raw/True.csv')
example_real = real_df['text'].sample(1).tolist()
print("---- Sample from True.csv ----")
classify_texts(example_real, threshold=0.7)

# 7) Verify on a fake‐news sample from the dataset
fake_df      = pd.read_csv('../data/raw/Fake.csv')
example_fake = fake_df['text'].sample(1).tolist()
print("---- Sample from Fake.csv ----")
classify_texts(example_fake, threshold=0.7)

---- Test provided examples ----
📰 Breaking news: Scientists discover a new species in the Amazon rainforest.
    → Predicted: fake (p_real=0.00, p_fake=1.00)

📰 Local health officials report that vaccination rates have reached a record high this quarter.
    → Predicted: fake (p_real=0.00, p_fake=1.00)

📰 In an exclusive interview, the celebrity reveals her secret to eternal youth.
    → Predicted: fake (p_real=0.00, p_fake=1.00)

---- Sample from True.csv ----
📰 WASHINGTON (Reuters) - The White House on Monday said the Justice Department was reviewing its options after a federal judge denied a request by President Donald Trump’s administration to enforce his ban on transgender troops while the government appeals an order blocking it. “The Department of Justice is currently reviewing the legal options to ensure the president’s directive is implemented,” White House spokeswoman Sarah Sanders told reporters. 
    → Predicted: real (p_real=1.00, p_fake=0.00)

---- Sample from Fake.csv --